In [1]:
import requests
import json

def call_insert_docs_into_pinecone(tickers, start_index, num_docs):
    url = "http://localhost:1000/api/insert_into_pinecone"

#     # prepare the data
#     data = {
#         'tickers': tickers,
#         'start_index': start_index,
#         'num_docs': num_docs,
#         'doc_type': '10-K'
#     }

#     # use json.dumps to convert dict to json string
#     response = requests.get(url, data=json.dumps(data), headers={'Content-Type': 'application/json'})
    
#     # print out the response for debugging
#     print(response.status_code)
#     print(response.text)
    
    # prepare the data
    data = {
        'tickers': tickers,
        'start_index': start_index,
        'num_docs': num_docs,
        'doc_type': '10-K'
    }

    # use json.dumps to convert dict to json string
    response = requests.get(url, data=json.dumps(data), headers={'Content-Type': 'application/json'})
    
    # print out the response for debugging
    print(response.status_code)
#     print(response.text)




In [12]:
call_insert_docs_into_pinecone(['TOST'], 0, 200)


500


In [23]:
import pinecone as pinecone_client

pinecone_client.init(api_key="8f191935-e17c-41f4-a31a-f5c96a86b9df", environment="eu-west1-gcp")
pinecone_index = pinecone_client.Index("meridian-public")


/opt/homebrew/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [84]:
import os
import openai
openai.api_key = os.getenv("OPENAI_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


{
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}


In [13]:
with open('/Users/alexanderwhedon/Downloads/kpi_edgar.json', 'r') as f:
    data = json.loads(f.read())

In [32]:
data[0]


{'id_': 'AAPL_10-K_0000320193-21-000105.txt',
 'segments': [{'id_': 15,
   'value': 'Net sales disaggregated by significant products and services for 2021, 2020 and 2019 were as follows (in millions): (1) Products net sales include amortization of the deferred value of unspecified software upgrade rights, which are bundled in the sales price of the respective product. (2) Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch and accessories. (3) Services net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other services. Services net sales also include amortization of the deferred value of services bundled in the sales price of certain products. (4) Includes $ 6.7 billion of revenue recognized in 2021 that was included in deferred revenue as of September 26, 2020, $ 5.0 billion of revenue recognized in 2020 that was included in deferred revenue as of September 28,

In [8]:
# -*- coding: utf-8 -*-
"""

SEC Filing Scraper
@author: AdamGetbags

"""

# import modules
import requests
import pandas as pd

# create request header
headers = {'User-Agent': "email@address.com"}

# get all companies data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )

# review response / keys
print(companyTickers.json().keys())

# format response to dictionary and get first key/value
firstEntry = companyTickers.json()['0']

# parse CIK // without leading zeros
directCik = companyTickers.json()['0']['cik_str']

# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(),
                                     orient='index')

# add leading zeros to CIK
companyData['cik_str'] = companyData['cik_str'].astype(
                           str).str.zfill(10)

# review data
print(companyData[:1])

company_data_dict = companyData.set_index('ticker')['cik_str'].to_dict()
company_data_dict

cik = company_data_dict['MSFT']

print(cik)

# get company specific filing metadata
filingMetadata = requests.get(
    f'https://data.sec.gov/submissions/CIK{cik}.json',
    headers=headers
    )

# review json 
print(filingMetadata.json().keys())
filingMetadata.json()['filings']
filingMetadata.json()['filings'].keys()
filingMetadata.json()['filings']['recent']
filingMetadata.json()['filings']['recent'].keys()

# dictionary to dataframe
allForms = pd.DataFrame.from_dict(
             filingMetadata.json()['filings']['recent']
             )

# review columns
allForms.columns
allForms[['accessionNumber', 'reportDate', 'form']].head(50)

# 10-Q metadata
allForms.iloc[11]

# get company facts data
companyFacts = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
    headers=headers
    )

#review data
companyFacts.json().keys()
companyFacts.json()['facts']
companyFacts.json()['facts'].keys()

# filing metadata
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding']
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding'].keys()
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding']['units']
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding']['units']['shares']
companyFacts.json()['facts']['dei'][
    'EntityCommonStockSharesOutstanding']['units']['shares'][0]

# concept data // financial statement line items
companyFacts.json()['facts']['us-gaap']
companyFacts.json()['facts']['us-gaap'].keys()

# different amounts of data available per concept
# companyFacts.json()['facts']['us-gaap']['AccountsPayable']
# companyFacts.json()['facts']['us-gaap']['Revenues']
# companyFacts.json()['facts']['us-gaap']['Assets']

# get company concept data
companyConcept = requests.get(
    (
    f'https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}'
     f'/dei/EntityCommonStockSharesOutstanding.json'
    ),
    headers=headers
    )

# review data
# companyConcept.json().keys()
# companyConcept.json()['units']
# companyConcept.json()['units'].keys()
# companyConcept.json()['units']['USD']
# companyConcept.json()['units']['USD'][0]

# parse assets from single filing
# companyConcept.json()['units']['USD'][0]['val']

# get all filings data 
assetsData = pd.DataFrame.from_dict((
               companyConcept.json()['units']['USD']))

# review data
assetsData.columns
assetsData.form

# get assets from 10Q forms and reset index
assets10Q = assetsData[assetsData.form == '10-Q']
assets10Q = assets10Q.reset_index(drop=True)

# plot 
assets10Q.plot(x='end', y='val')

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', 

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])


KeyError: 'USD'

In [3]:
cf = companyFacts.json()
cf.keys()

dict_keys(['cik', 'entityName', 'facts'])

In [11]:
g = [print(x) for x in cf['facts']['us-gaap'].keys()]

AccountsPayableCurrent
AccountsReceivableNet
AccountsReceivableNetCurrent
AccountsReceivableNetNoncurrent
AccruedIncomeTaxesCurrent
AccruedIncomeTaxesNoncurrent
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment
AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax
AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax
AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax
AccumulatedOtherComprehensiveIncomeLossNetOfTax
AcquiredFiniteLivedIntangibleAssetAmount
AcquiredFiniteLivedIntangibleAssetWeightedAverageUsefulLife
AdvertisingExpense
AllocatedShareBasedCompensationExpense
AllowanceForDoubtfulAccountsReceivableCurrent
AmortizationOfIntangibleAssets
AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount
AssetImpairmentCharges
Assets
AssetsCurrent
AssetsFairValueDisclosureRecurring
AvailableForSaleDebtSecuritiesAmortizedCostBasis
AvailableForSa

In [ ]:
cf['']

In [3]:
companyData

,cik_str,ticker,title
0,0000320193,AAPL,Apple Inc.
1,0000789019,MSFT,MICROSOFT CORP
2,0001652044,GOOGL,Alphabet Inc.
3,0001018724,AMZN,AMAZON COM INC
4,0001045810,NVDA,NVIDIA CORP
...,...,...,...
8979,0001716951,OCCIO,"OFS Credit Company, Inc."
8980,0001716951,OCCIN,"OFS Credit Company, Inc."
8981,0001360214,HROWM,"HARROW HEALTH, INC."
8982,0001496254,LTCFX,Alternative Strategies Fund


In [63]:
# cf = companyFacts.json()
g = [print(x) for x in cf['facts']['us-gaap'].keys()]


AccountsPayableCurrent
AccountsReceivableNet
AccountsReceivableNetCurrent
AccountsReceivableNetNoncurrent
AccruedIncomeTaxesCurrent
AccruedIncomeTaxesNoncurrent
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment
AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax
AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax
AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax
AccumulatedOtherComprehensiveIncomeLossNetOfTax
AcquiredFiniteLivedIntangibleAssetAmount
AcquiredFiniteLivedIntangibleAssetWeightedAverageUsefulLife
AdvertisingExpense
AllocatedShareBasedCompensationExpense
AllowanceForDoubtfulAccountsReceivableCurrent
AmortizationOfIntangibleAssets
AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount
AssetImpairmentCharges
Assets
AssetsCurrent
AssetsFairValueDisclosureRecurring
AvailableForSaleDebtSecuritiesAmortizedCostBasis
AvailableForSa

In [71]:
companyConcept.json()

{'cik': 789019,
 'taxonomy': 'dei',
 'tag': 'EntityCommonStockSharesOutstanding',
 'label': 'Entity Common Stock, Shares Outstanding',
 'description': "Indicate number of shares or other units outstanding of each of registrant's classes of capital or common stock or other ownership interests, if and as stated on cover of related periodic report. Where multiple classes or units exist define each class/interest by adding class of stock items such as Common Class A [Member], Common Class B [Member] or Partnership Interest [Member] onto the Instrument [Domain] of the Entity Listings, Instrument.",
 'entityName': 'MICROSOFT CORPORATION',
 'units': {'shares': [{'end': '2009-10-19',
    'val': 8879121378,
    'accn': '0001193125-09-212454',
    'fy': 2010,
    'fp': 'Q1',
    'form': '10-Q',
    'filed': '2009-10-23',
    'frame': 'CY2009Q3I'},
   {'end': '2010-01-25',
    'val': 8770460922,
    'accn': '0001193125-10-015598',
    'fy': 2010,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '

In [43]:
company_data_dict = companyData.set_index('ticker')['cik_str'].to_dict()
company_data_dict


{'AAPL': '0000320193',
 'MSFT': '0000789019',
 'GOOGL': '0001652044',
 'AMZN': '0001018724',
 'NVDA': '0001045810',
 'META': '0001326801',
 'BRK-B': '0001067983',
 'TSLA': '0001318605',
 'V': '0001403161',
 'LLY': '0000059478',
 'TSM': '0001046179',
 'UNH': '0000731766',
 'LVMUY': '0000824046',
 'JNJ': '0000200406',
 'JPM': '0000019617',
 'XOM': '0000034088',
 'WMT': '0000104169',
 'LTMAY': '0001047716',
 'NVO': '0000353278',
 'SPY': '0000884394',
 'MA': '0001141391',
 'PG': '0000080424',
 'AVGO': '0001730168',
 'HD': '0000354950',
 'ORCL': '0001341439',
 'CVX': '0000093410',
 'ASML': '0000937966',
 'MRK': '0000310158',
 'ABBV': '0001551152',
 'KO': '0000021344',
 'BABA': '0001577552',
 'PEP': '0000077476',
 'COST': '0000909832',
 'BAC': '0000070858',
 'ADBE': '0000796343',
 'TM': '0001094517',
 'NVS': '0001114448',
 'AZN': '0000901832',
 'CSCO': '0000858877',
 'TMO': '0000097745',
 'MCD': '0000063908',
 'SHEL': '0001306965',
 'CRM': '0001108524',
 'PFE': '0000078003',
 'FMX': '0001061

In [77]:
d.keys()

dict_keys(['instance', 'std_ref', 'version'])

In [82]:
d['instance']['tost-20210930.htm'].keys()

dict_keys(['axisCustom', 'axisStandard', 'contextCount', 'dts', 'elementCount', 'entityCount', 'hidden', 'keyCustom', 'keyStandard', 'memberCustom', 'memberStandard', 'nsprefix', 'nsuri', 'report', 'segmentCount', 'tag', 'unitCount'])

In [84]:
d['instance']['tost-20210930.htm']['memberCustom']

27

In [85]:
d['instance']['tost-20210930.htm']

{'axisCustom': 0,
 'axisStandard': 32,
 'contextCount': 334,
 'dts': {'calculationLink': {'local': ['tost-20210930_cal.xml']},
  'definitionLink': {'local': ['tost-20210930_def.xml']},
  'inline': {'local': ['tost-20210930.htm']},
  'labelLink': {'local': ['tost-20210930_lab.xml']},
  'presentationLink': {'local': ['tost-20210930_pre.xml']},
  'schema': {'local': ['tost-20210930.xsd'],
   'remote': ['https://xbrl.fasb.org/srt/2021/elts/srt-2021-01-31.xsd',
    'http://www.xbrl.org/2003/xbrl-instance-2003-12-31.xsd',
    'http://www.xbrl.org/2003/xbrl-linkbase-2003-12-31.xsd',
    'http://www.xbrl.org/2003/xl-2003-12-31.xsd',
    'http://www.xbrl.org/2003/xlink-2003-12-31.xsd',
    'https://www.xbrl.org/dtr/type/2020-01-21/types.xsd',
    'https://www.xbrl.org/2020/extensible-enumerations-2.0.xsd',
    'http://www.xbrl.org/2005/xbrldt-2005.xsd',
    'http://www.xbrl.org/2006/ref-2006-02-27.xsd',
    'https://xbrl.fasb.org/srt/2021/elts/srt-types-2021-01-31.xsd',
    'https://xbrl.sec.go

In [1]:
# -*- coding: utf-8 -*-
"""

SEC Filing Scraper
@author: AdamGetbags

"""

# import modules
import requests
import pandas as pd

# create request header
headers = {'User-Agent': "email@address.com"}

# get all companies data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )

# review response / keys
print(companyTickers.json().keys())

# format response to dictionary and get first key/value
firstEntry = companyTickers.json()['0']

# parse CIK // without leading zeros
directCik = companyTickers.json()['0']['cik_str']

# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(),
                                     orient='index')

# add leading zeros to CIK
companyData['cik_str'] = companyData['cik_str'].astype(
                           str).str.zfill(10)

# review data
print(companyData[:1])

company_data_dict = companyData.set_index('ticker')['cik_str'].to_dict()
company_data_dict

cik = company_data_dict['MSFT']

print(cik)

# # get company specific filing metadata
filingMetadata = requests.get(
    f'https://data.sec.gov/submissions/CIK{cik}.json',
    headers=headers
    )

# review json 
print(filingMetadata.json().keys())
filingMetadata.json()['filings']
filingMetadata.json()['filings'].keys()
filingMetadata.json()['filings']['recent']
filingMetadata.json()['filings']['recent'].keys()

# dictionary to dataframe
allForms = pd.DataFrame.from_dict(
             filingMetadata.json()['filings']['recent']
             )

# review columns
allForms.columns
allForms[['accessionNumber', 'reportDate', 'form']].head(50)

# 10-Q metadata
allForms.iloc[11]

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', 

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])


accessionNumber                                       0001062993-23-013583
filingDate                                                      2023-06-14
reportDate                                                      2023-06-13
acceptanceDateTime                                2023-06-14T18:10:55.000Z
act                                                                       
form                                                                     4
fileNumber                                                                
filmNumber                                                                
items                                                                     
size                                                                  8227
isXBRL                                                                   0
isInlineXBRL                                                             0
primaryDocument                                       xslF345X04/form4.xml
primaryDocDescription    

In [4]:
allForms[allForms['form'] == '10-Q'].reset_index(drop = True)

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000950170-23-014423,2023-04-25,2023-03-31,2023-04-25T16:07:43.000Z,34,10-Q,001-37845,23844923,,32928186,1,1,msft-20230331.htm,10-Q
1,0001564590-23-000733,2023-01-24,2022-12-31,2023-01-24T16:34:20.000Z,34,10-Q,001-37845,23548555,,22217673,1,1,msft-10q_20221231.htm,10-Q
2,0001564590-22-035087,2022-10-25,2022-09-30,2022-10-25T16:08:55.000Z,34,10-Q,001-37845,221329593,,18509528,1,1,msft-10q_20220930.htm,10-Q
3,0001564590-22-015675,2022-04-26,2022-03-31,2022-04-26T16:08:55.000Z,34,10-Q,001-37845,22854390,,22314465,1,1,msft-10q_20220331.htm,10-Q
4,0001564590-22-002324,2022-01-25,2021-12-31,2022-01-25T16:09:04.000Z,34,10-Q,001-37845,22554019,,21933276,1,1,msft-10q_20211231.htm,10-Q
5,0001564590-21-051992,2021-10-26,2021-09-30,2021-10-26T16:11:30.000Z,34,10-Q,001-37845,211348546,,18484545,1,1,msft-10q_20210930.htm,10-Q
6,0001564590-21-020891,2021-04-27,2021-03-31,2021-04-27T16:09:38.000Z,34,10-Q,001-37845,21859007,,22586167,1,1,msft-10q_20210331.htm,10-Q
7,0001564590-21-002316,2021-01-26,2020-12-31,2021-01-26T16:09:34.000Z,34,10-Q,001-37845,21554140,,21735875,1,1,msft-10q_20201231.htm,10-Q
8,0001564590-20-047996,2020-10-27,2020-09-30,2020-10-27T16:21:58.000Z,34,10-Q,001-37845,201264355,,18132017,1,1,msft-10q_20200930.htm,10-Q
9,0001564590-20-019706,2020-04-29,2020-03-31,2020-04-29T16:11:11.000Z,34,10-Q,001-37845,20830395,,21306500,1,1,msft-10q_20200331.htm,10-Q


In [6]:
assetsData

NameError: name 'assetsData' is not defined